# QT User Meeting Tutorial 1 - Demo 2
## Ramsey experiment

Copyright (C) 2022 Zurich Instruments

This software may be modified and distributed under the terms of the MIT license. See the LICENSE file for details.

# 0. General Imports and Definitions

## 0.1 Python Imports

In [ ]:
%config IPCompleter.greedy=True

# convenience import for all QCCS software functionality
from qccs.simple import *
# pulse sheet viewer generates a html file that displays the pulse sequence 
from qccs.pulse_sheet_viewer.pulse_sheet_viewer import show_pulse_sheet

# additional imports for plotting 
import matplotlib.pyplot as plt
import numpy as np

## 0.2 Definition for Result Plotting

In [ ]:
# 2D plot
def plot_result_2d(results, handle):
    acquired_data = results.get_data(handle)
    axis_grid = results.get_axis(handle)[0]
    axis_name = results.get_axis_name(handle)[0]
    
    plt.plot(axis_grid, np.absolute(acquired_data))
    plt.xlabel(axis_name)
    plt.ylabel(handle)

# 3D plot
def plot_result_3d(results, handle):
    acquired_data = results.get_data(handle)
    y_axis_grid = results.get_axis(handle)[0]
    y_axis_name = results.get_axis_name(handle)[0]
    x_axis_grid = results.get_axis(handle)[1]
    x_axis_name = results.get_axis_name(handle)[1]
    
    X, Y = np.meshgrid(x_axis_grid, y_axis_grid)
    
    ax = plt.axes(projection='3d')
    ax.plot_wireframe(X, Y, np.absolute(acquired_data))
    ax.set_xlabel(x_axis_name)
    ax.set_ylabel(y_axis_name)
    ax.set_zlabel(handle)
    
    plt.figure() # Create new dummy figure to ensure no side effects of the current 3D figure

# 1. Define Device Setup and Calibration

## 1.1 Define a Device Setup

Descriptor contains all information on instruments used, internal connections between instruments as well as wiring to the experiment

In [ ]:
descriptor="""\
instrument_list:
  HDAWG:
  - address: DEV8297
    uid: device_hdawg
  SHFSG:
  - address: DEV12127
    uid: device_shfsg
  SHFQA:
  - address: DEV12121    
    uid: device_shfqa
  PQSC:  
  - address: DEV10064
    uid: device_pqsc
connections:
  device_hdawg:
    - rf_signal: q0/flux_line
      ports: [SIGOUTS/0]        
  device_shfsg:    
    - iq_signal: q0/drive_line
      ports: SGCHANNELS/0/OUTPUT        
  device_shfqa:    
    - iq_signal: q0/measure_line
      ports: QACHANNELS/0/OUTPUT        
    - acquire_signal: q0/acquire_line
      ports: QACHANNELS/0/INPUT
  device_pqsc:
    - to: device_shfsg
      port: ZSYNCS/1
    - to: device_shfqa
      port: ZSYNCS/2
    - to: device_hdawg
      port: ZSYNCS/0
"""

## 1.2 Define Calibration Settings

Modify the calibration on the device setup with known parameters for qubit control and readout - qubit control and readout frequencies, mixer calibration corrections

In [ ]:
# functions that modifies the calibration on a given device setup
def calibrate_devices(device_setup):
    ## qubit 0
    # calibration setting for drive line for qubit 0
    device_setup.logical_signal_groups["q0"].logical_signals["drive_line"].calibration = SignalCalibration(
        # oscillator settings - frequency and type of oscillator used to modulate the pulses applied through this signal line
        oscillator=Oscillator(
            uid="drive_q0_osc",
            frequency=1e8,
            modulation_type=ModulationType.HARDWARE),
        local_oscillator=Oscillator(
            "lo_shfsg",
            frequency=1e9
        ),
        range = 5,
        # global and static delay of logical signal line: use to align pulses and compensate skew
        port_delay=0,       # applied to corresponding instrument node, bound to hardware limits
        delay_signal=0,     # inserted in sequencer code, bound to waveform granularity
    )
    # calibration setting for flux line for qubit 0
    device_setup.logical_signal_groups["q0"].logical_signals["flux_line"].calibration = SignalCalibration(
        oscillator=Oscillator(
            uid="flux_q0_osc",
            frequency=1e8,
            modulation_type=ModulationType.HARDWARE),
        # global and static delay of logical signal line: use to align pulses and compensate skew
        port_delay=0,       # applied to corresponding instrument node, bound to hardware limits
        delay_signal=0,     # inserted in sequencer code, bound to waveform granularity
    )
    # calibration setting for readout pulse line for qubit 0
    device_setup.logical_signal_groups["q0"].logical_signals["measure_line"].calibration = SignalCalibration(
        oscillator = Oscillator(
            uid="measure_q0_osc",
            frequency=1e8,
            modulation_type=ModulationType.SOFTWARE),
        local_oscillator=Oscillator(
            "lo_shfqa",
            frequency=1.2e9
        ),
        range = 5,
        delay_signal=0,     # inserted in sequencer code, bound to waveform granularity
        )
    # calibration setting for data acquisition line for qubit 0
    device_setup.logical_signal_groups["q0"].logical_signals["acquire_line"].calibration = SignalCalibration(
        oscillator=Oscillator(
            uid="acquire_osc",
            frequency=1e8,
            modulation_type=ModulationType.SOFTWARE),
        local_oscillator=Oscillator(
            "lo_shfqa",
            frequency=1.2e9
        ),
        range = 5,
        # delays the start of integration in relation to the start of the readout pulse to compensate for signal propagation time
        port_delay=10e-9,   # applied to corresponding instrument node, bound to hardware limits
        delay_signal=0,     # inserted in sequencer code, bound to waveform granularity
    )

## 1.3 Create Device Setup and Apply Calibration Settings

In [ ]:
# Function returning a calibrated device setup 
def create_device_setup():
    device_setup = DeviceSetup.from_descriptor(
        descriptor,
        server_host="localhost",    # ip address of the LabOne dataserver used to communicate with the instruments
        server_port="8004",             # port number of the dataserver - default is 8004
        setup_name="my_QCCS_setup",     # setup name
    ) 
    calibrate_devices(device_setup)
    return device_setup

# create device setup
device_setup = create_device_setup()

In [ ]:
# use emulation mode - change, if running on hardware
use_emulation=True

# 2. Ramsey Experiment

Sweep the delay between two slightly detuned pi/2 pulses to determine the qubit dephasing time as well as fine calibration its excited state frequency

## 2.1 Define the Experiment

In [ ]:
## define pulses

# qubit drive pulse - use amplitude calibrated by amplitude Rabi experiment
x90 = pulse_library.gaussian(uid="x90", length=100e-9, amplitude=1)
# readout drive pulse
readout_pulse = pulse_library.const(
    uid="readout_pulse", length=400e-9, amplitude=1.0
)
# readout integration weights
readout_weighting_function = pulse_library.const(
    uid="readout_weighting_function", length=400e-9, amplitude=1.0
)

In [ ]:
# set up sweep parameter - delay between pi/2 pulses
start = 0.0
stop = 1000e-9
count = 10

sweep_parameter = LinearSweepParameter(uid="delay", start=start, stop=stop, count=count)
amplitude_sweep = LinearSweepParameter(uid="amplitude", start=0, stop=1, count=count)

# number of averages
average_exponent = 1  # used for 2^n averages, n=average_exponent

# Create Experiment
exp = Experiment(
    uid="Ramsey",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("measure"),
        ExperimentSignal("acquire"),
    ],
)
## experimental pulse sequence
# outer loop - real-time, cyclic averaging in standard integration mode
with exp.acquire_loop_rt(uid="shots", count=pow(2, average_exponent),
    averaging_mode=AveragingMode.CYCLIC, acquisition_type=AcquisitionType.INTEGRATION
    ):
    # inner loop - real-time sweep over delay between qubit pulses
    with exp.sweep(uid="sweep", parameter=[sweep_parameter, amplitude_sweep]):
        # qubit drive pulses - use right-aligned, constant length section to optimize overall experimental sequence
        with exp.section(uid="qubit_excitation", length=stop+2*x90.length, alignment=SectionAlignment.RIGHT):
            exp.play(signal="drive", pulse=x90)
            exp.delay(signal="drive", time=sweep_parameter)
            exp.play(signal="drive", pulse=x90, amplitude=amplitude_sweep)
        # qubit readout pulse and data acquisition
        with exp.section(uid="qubit_readout"):
            exp.reserve(signal="drive")
            # add a delay before the readout pulse
            exp.delay(signal="measure", time=10e-9)
            exp.delay(signal="acquire", time=10e-9)
            # play readout pulse
            exp.play(signal="measure", pulse=readout_pulse)
            # signal data acquisition
            exp.acquire(
                signal="acquire",
                handle="ac_0",
                kernel=readout_weighting_function,
            )
        # relax time after readout - for signal processing and qubit relaxation to groundstate
        with exp.section(uid="relax"):
            exp.delay(signal="measure", time=1e-6)

In [ ]:
# define signal maps for qubit 0
map_q0 = {
    "drive": device_setup.logical_signal_groups["q0"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q0"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q0"].logical_signals["acquire_line"],
}

## 2.2 Run the Experiment and Plot the Measurement Results and Pulse Sequence

In [ ]:
# set signal map to qubit 0
exp.set_signal_map(map_q0)

# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run experiment on qubit 0
session.run_all(exp)

# 3. Pulse Sheet Viewer

In [ ]:
# plot output signals with the integrated plotter
Plotter.plot(session.results.output_signals)

In [ ]:
# use pulse sheet viewer to display the pulse sequence
compiled_exp = session.compiled_experiment
show_pulse_sheet("Ramsey", compiled_exp)